In [ ]:
import pandas as pd
import re
import json
from pandas.io.json import json_normalize
import requests
from bs4 import BeautifulSoup

In [ ]:
###Scrape a complete part list###

#Get content and find all a[href]
def getHref(url):
    soupOverview = BeautifulSoup(requests.get(url).text, 'html.parser')
    urls = soupOverview.select('#content_block')[0].findAll('a', href=True)
    return urls

#Settings
tisUrlOverview = ""
filename = tisUrlOverview.split("parts-catalog/")[1].split("/browse")[0].replace('/', "_").replace('.', "").replace('-', "_")
baseUrl = ""

#Get all 1st level category URLs and parse them to object
categories = []
for e in getHref(tisUrlOverview):
    if "browse" in e['href']: 
        categories.append({
            "title" : e.text.strip(),
            "url" : baseUrl+e['href'],
            "subcategories" : [], 
        })
print(len(categories), "categories fetched!")

#Get all subcategories URLs
scCounter = 0
for i in range(0, len(categories)):
    for e in getHref(categories[i].get('url')):
        if "browse" in e['href']: 
            categories[i]["subcategories"].append({
                "title" : e.text.strip(),
                "url" : baseUrl+e['href'], 
                "parts" : [],     
            })
            scCounter += 1
print(str(scCounter), "subcategories fetched!")

#For all Categories
parts = []
part = {}
partVariation = ""
for i in range(0, len(categories)):
    for z in range(0, len(categories[i].get('subcategories'))-1):
        url = categories[i].get('subcategories')[z].get('url')
        partsTable = BeautifulSoup(requests.get(url).text, 'html.parser').select('#parts_table')[0].find('tbody')
    
        #Parse parts page
        part = {}
        partVariation = ""
        for e in partsTable.findAll('tr', recursive=False)[0:]:

            #Get name of the part from "title_row" 
            if e.select('.title_row') and len(e.select('.small_text')[0].text) > 0:
                partName = e.select('.small_text')[0].text
                if len(e.select('.small_text')) > 1:
                    partVariation = e.select('.small_text')[1].text 

            #Get other Columns
            if (not e.select('.title_row') and len(e.text.strip())>0 and e.get("id") != "accent_tr"):
                part = {}
                part["name"] = partName 
                part["id"] = e.select('.clipboard_part')[0].get('value')
                part["note"] = e.select('.clipboard_part')[0].findNext('td', {"class": "part_td"}).text.strip().replace('\n', ' ')
                if len(e.select(".add_td"))> 0 and 'Für Fahrzeuge mit' in e.select(".add_td")[0].text:
                    partVariation = e.select(".add_td")[0].select("div.mobile.only")[0].select("div.drop_info.hide")[0].text

                if len(partVariation) > 0 and 'Für Fahrzeuge mit' in partVariation:
                    part["variation"] = re.split('Für Fahrzeuge mit', partVariation)[1].replace('\n','').replace('', '').replace("Ja", "Ja ").replace("Nein", "Nein ")
                else:
                    part["variation"] = ""
                part["amount"] = e.select("td.light.part_td.small.center_text")[0].text.strip().replace('\n', ' ')    
                part["category"] = categories[i].get('title')
                part["categoryURL"] = categories[i].get('url')
                part["subcategory"] = categories[i].get('subcategories')[z].get('title')
                part["subcategoryURL"] = categories[i].get('subcategories')[z].get('url')
                parts.append(part)

            #Reset new Part
            if (e.get("id") == "accent_tr"):
                partVariation = ""
                partName = ""
        print(categories[i].get('subcategories')[z].get('title'), "succesfull crawled")
    print(categories[i].get('title'), "succesfull crawled")
    
#Export to DataFrame
data = json.dumps(parts)
json_data = json.loads(data)
df = json_normalize(json_data)
df.to_csv(filename + '.csv', index = false)

In [ ]:
#Setup
cat_compare = "Bremsen"
#subcat_compare = "Hinterradbremse-Bremsscheibe"

#Chose Cars and Read Files
df_c_initial = pd.read_csv('E85_25_M54.csv')
df_c_compare = pd.read_csv('E85_30_M54.csv')

#Chose Category to Compare and Trim DF
df_c_initial = df_c_initial[df_c_initial.category == cat_compare]
df_c_compare = df_c_compare[df_c_compare.category == cat_compare]

#Chose Subcategory to Compare and Trim DF
if subcat_compare:
    df_c_initial = df_c_initial[df_c_initial.subcategory == subcat_compare]
    df_c_compare = df_c_compare[df_c_compare.subcategory == subcat_compare]

#Compare Both DF
df_compared = pd.DataFrame()
for tn in df_c_compare.id:
    if len(df_c_initial[df_c_initial.id == tn]) == 0:
        df_compared = df_compared.append(df_c_compare[df_c_compare.id == tn])
df_compared